<center> <h1>PREDICIÓN DEL SUEÑO CON MLP</h1> </center>
<center> <h4>Noemi González, Roberto Holgado y Carmen Plaza seco</h4> </center>


###### Indice

En este jupyter notebook se lleva a cabo la construcción de un clasificador MLP de salida binaria para predecir el el estado de vigilia o sueño ligero y  el de sueño profundo.

* **1. Cargar datos Train-Test**
  * **1.1 Preparación datos de entrada y salida**
* **2. Clasificador MLP binario**
   * **2.1 Creación del modelo y selección parámetros libres**
   * **2.3 Evaluación del modelo**

IMPORTAR MÓDULOS

In [18]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import svm
import sklearn

## 1. CARGAR DATOS TRAIN-TEST

Se cargan los datos de train y test generados en los jupyter notebooks 'Nombre de los jupyter', en los que procesan los datos y se obtienen las características que se utilizan para entrenar los clasificadores. 

En el siguiente apartado se preparan los datos para seleccionar una salida binaria o multiclase según el clasificador que se vaya a utilizar.

### 2.1 Preparación datos de entrada y salida

In [19]:
Train= pd.read_csv('Train.csv')

In [20]:
Train

,Unnamed: 0,Min_hr,Max_hr,Mean_hr,Men_acc_sqrt,Min_acc_x,Max_acc_x,Mean_acc_x,Min_acc_y,Max_acc_y,Mean_acc_y,Min_acc_z,Max_acc_z,Mean_acc_z,Etiquetas,Etiquetas binarias,ID_P
0,0,61.0,63.0,61.83,1.01,-0.13,-0.10,-0.11,0.97,1.00,0.98,-0.23,-0.15,-0.17,-1.0,0.0,22.0
1,98,63.0,63.0,63.00,1.00,0.50,0.51,0.51,-0.26,-0.24,-0.25,-0.83,-0.82,-0.82,3.0,1.0,22.0
2,196,66.0,69.0,67.00,1.00,0.67,0.68,0.67,0.44,0.45,0.45,-0.60,-0.59,-0.59,2.0,1.0,22.0
3,295,69.0,75.0,71.67,1.00,0.31,0.33,0.32,0.19,0.21,0.20,-0.94,-0.92,-0.93,3.0,1.0,22.0
4,393,65.0,65.0,65.00,1.00,-0.44,-0.42,-0.43,0.19,0.20,0.19,-0.89,-0.88,-0.88,2.0,1.0,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,503,62.0,68.0,66.00,1.00,-0.28,-0.27,-0.28,-0.23,-0.22,-0.22,-0.94,-0.93,-0.94,2.0,1.0,1.0
206,596,74.0,79.0,76.80,1.00,-0.04,-0.03,-0.03,-0.01,-0.01,-0.01,0.99,1.00,1.00,5.0,0.0,1.0
207,688,70.0,78.0,73.00,1.00,-0.37,-0.11,-0.18,0.38,0.83,0.75,-0.90,-0.54,-0.61,0.0,0.0,1.0
208,781,74.0,78.0,75.67,1.00,0.02,0.03,0.03,-0.14,-0.11,-0.11,-1.00,-0.99,-0.99,5.0,0.0,1.0


In [21]:
#Train
Train= pd.read_csv('Train.csv')
#.drop(['Unnamed: 0'], axis = 1) a mi me los lee sin unnamed
Test= pd.read_csv('Test.csv')
#.drop(['Unnamed: 0'], axis = 1)

In [22]:
X_train=Train.drop(['Etiquetas', 'Etiquetas binarias','ID_P'], axis = 1) 
y_train_bin=Train['Etiquetas binarias']
y_train_multi=Train['Etiquetas']

X_test=Test.drop(['Etiquetas', 'Etiquetas binarias','ID_P'], axis = 1)
y_test_bin=Test['Etiquetas binarias']
y_test_multi=Test['Etiquetas']

## 2. CLASIFICADOR MLP BINARIO

Se ha utilizado un perceptrón multicapa para clasificar la vigilia o sueño ligero del sueño profundo. Es decir, se ha construido un clasificador con salida binaria.

### 2.1  Creación del modelo y selección parámetros libres

Se proponen posibles valores de neuronas en la capa oculta para ver con cuál de ellos se obtienen las mejores prestaciones para el modelo. Se crea una tabla que recoge la pérdidas para cada valor del número de neuronas y aquel que tenga menos pérdida es el que se utilizará para el conjunto de test.

In [23]:
# Número de neuronas de la capa oculta

n_hidden=range(4,20,3) 

In [24]:
# Tabla para recoger las prestaciones en train 
columns  = ['Loss','Score']
comp_train = pd.DataFrame(columns=columns)

for i in n_hidden:
    
    print('\033[4;32m' + 'MLP clasiffier with',i,'neurons in the hidden layer' + '\033[0m')

    model=MLPClassifier(activation='relu', batch_size=50, hidden_layer_sizes=(i,), learning_rate='constant',
        learning_rate_init=0.1, max_iter=30, shuffle=True, solver='sgd', tol=0.01, early_stopping=True,
        validation_fraction=0.2, verbose=True, n_iter_no_change=5)
    
    model.fit(X_train, y_train_bin)
    
    loss=model.loss_  
    comp_train = comp_train.set_value(i,('Loss','Score'),(loss,'Verbose'))

MLP clasiffier with 4 neurons in the hidden layer
Iteration 1, loss = inf
Validation score: 0.666667
Iteration 2, loss = 2.69494662
Validation score: 0.666667
Iteration 3, loss = 4.84988040
Validation score: 0.666667
Iteration 4, loss = 6.70359845
Validation score: 0.666667
Iteration 5, loss = 8.11071799
Validation score: 0.666667
Iteration 6, loss = 9.11338306
Validation score: 0.666667
Iteration 7, loss = 9.80611502
Validation score: 0.666667
Validation score did not improve more than tol=0.010000 for 5 consecutive epochs. Stopping.
MLP clasiffier with 7 neurons in the hidden layer
Iteration 1, loss = inf
Validation score: 0.666667
Iteration 2, loss = 8.30962724
Validation score: 0.666667
Iteration 3, loss = 16.35455890
Validation score: 0.666667
Iteration 4, loss = 23.27984453
Validation score: 0.666667
Iteration 5, loss = 28.53923446
Validation score: 0.666667
Iteration 6, loss = 32.29575516
Validation score: 0.666667
Iteration 7, loss = 34.88874061
Validation score: 0.666667
Valid

/Applications/anaconda3/envs/cuarto/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()
/Applications/anaconda3/envs/cuarto/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()
/Applications/anaconda3/envs/cuarto/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()
/Applications/anaconda3/envs/cuarto/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()
/Applications/anaconda3/envs

In [25]:
comp_train

,Loss,Score
4,9.80612,Verbose
7,34.8887,Verbose
10,26.4811,Verbose
13,13.2268,Verbose
16,64.8208,Verbose
19,2.90022,Verbose


Las medidas de prestación elegidas para la elección de una topología han sido el validation score y la pérdida. Ambas se pueden ver al aplicar el fit del modelo, gracias al parámetro verbose=True. La pérdida la hemos añadido a la siguiente tabla pero la accuracy de cada epoch para el conjunto de validación hay que verla en la celda anterior. Según estos resultados, la mejor topología de MLP sería la de **n_hidden_1 con 5 neuronas** en la capa oculta, ya que el par (loss,val_score) es el mejor de los 4.
En todos los casos el valor 'validation score' está entorno a 0.66, pero el número de neuronas de menor pérdida es 5.

¿Cómo sabíamos si estaba memorizando o no?

### 2.2 Evaluación del modelo

Testeamos el modelo con el valor de número de neuronas en la capa ocula que ofrece la menor pérdida (número neuronas = 5)

In [26]:
columns  = ['Accuracy','Recall','F1-score']
comp_test = pd.DataFrame(columns=columns)

print('\033[4;32m' + 'MLP clasiffier with 4 neurons in the hidden layer' + '\033[0m')

model=MLPClassifier(activation='relu', batch_size=50, hidden_layer_sizes=4, learning_rate='constant',
    learning_rate_init=0.1, max_iter=30, shuffle=True, solver='sgd', tol=0.01, early_stopping=True,
    validation_fraction=0.2, verbose=True, n_iter_no_change=5)

model.fit(X_train,y_train_bin)

predictions = model.predict(X_test)
cr=classification_report(y_test_bin,predictions,output_dict=True)
acc,rec,f1=cr['weighted avg']['precision'],cr['weighted avg']['recall'],cr['weighted avg']['f1-score']

comp_test = comp_test.set_value('4', ('Accuracy','Recall','F1-score'),(acc,rec,f1))

MLP clasiffier with 4 neurons in the hidden layer
Iteration 1, loss = inf
Validation score: 0.333333
Iteration 2, loss = 0.74467650
Validation score: 0.666667
Iteration 3, loss = 0.67670209
Validation score: 0.666667
Iteration 4, loss = 0.65841367
Validation score: 0.666667
Iteration 5, loss = 0.65347744
Validation score: 0.666667
Iteration 6, loss = 0.65374430
Validation score: 0.666667
Iteration 7, loss = 0.65376700
Validation score: 0.666667
Iteration 8, loss = 0.65252473
Validation score: 0.666667
Validation score did not improve more than tol=0.010000 for 5 consecutive epochs. Stopping.


/Applications/anaconda3/envs/cuarto/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Applications/anaconda3/envs/cuarto/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()


In [27]:
comp_test

,Accuracy,Recall,F1-score
4,0.4489,0.67,0.537605


In [28]:
print("================Classification Report=================")
print(cr)

================Classification Report=================
{'0.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 33}, '1.0': {'precision': 0.67, 'recall': 1.0, 'f1-score': 0.8023952095808384, 'support': 67}, 'accuracy': 0.67, 'macro avg': {'precision': 0.335, 'recall': 0.5, 'f1-score': 0.4011976047904192, 'support': 100}, 'weighted avg': {'precision': 0.4489, 'recall': 0.67, 'f1-score': 0.5376047904191618, 'support': 100}}
